In [ ]:
import os
import ast
import re
import operator as op

from datetime import datetime, timedelta
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
#from langchain.tools import tool
from langchain.agents import Tool, create_tool_calling_agent, AgentExecutor, tool
from typing import List, Optional
from pydantic import BaseModel

load_dotenv()

In [ ]:
# --- Tools ---

# --- 1. Mathematical expressions ---
operators = {
    ast.Add: op.add,
    ast.Div: op.truediv,
    ast.Mult: op.mul,
    ast.Pow: op.pow,
    ast.Sub: op.sub,
    ast.USub: op.neg,
}

def eval_check(node):
    """
    Safely evaluate an expression node.
    """
    if isinstance(node, ast.Constant):
        if isinstance(node.value, (int, float)):
            return node.value
        else:
            raise ValueError(f"Only intergers or float values are allowed. Unsupported type: {type(node.value)}")
    elif isinstance(node, ast.BinOp):
        left = eval_check(node.left)
        right = eval_check(node.right)
        return operators[type(node.op)](left, right)
    elif isinstance(node, ast.UnaryOp):
        return operators[type(node.op)](eval_check(node.operand))
    else:
        raise TypeError(f"Unsupported type: {type(node)}")
    
def dangerous_code_detection(expr: str) -> bool:
    """
    Check if the code contains dangerous injections.
    """
    return bool(re.search(r"[a-zA-Z_]", expr))

def eval_expression(expression: str) -> str:
    """
    Evaluate a mathematical expression safely.
    """
    if dangerous_code_detection(expression):
        return "Error: Invalid characters in expression."
    try:
        parsed = ast.parse(expression, mode='eval')
        result = eval_check(parsed.body)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"
    
# --- 2. Date difference calculator ---
def days_passed_calculation(input: str, format_str="%Y-%m-%d") -> str:
    try:
        user_date = datetime.strptime(input, format_str)
    except ValueError:
        print(f"Error: Could not parse '{input}' with format '{format_str}.")
        return None
    
    current_date = datetime.today()

    time_difference: timedelta = current_date - user_date

    days_passed = time_difference.days

    return f"{days_passed} days have passed since {user_date.date()}." # str(days_passed)

# --- Fahrenheit to Celsius conversion tool ---
#def fahrenheit_to_celsius(input: str) -> str:
    """
    Convert Fahrenheit to Celsius.
    """
    try:
        match = re.search(r"[-+]?\d*\.?\d+", input)
        if not match:
            return "Error: No valid number found."
        temp_in_fahrenheit = float(match.group())
        temp_in_celsius = (temp_in_fahrenheit - 32) * 5.0/9.0
        return f"{temp_in_fahrenheit}°F is {temp_in_celsius:.2f}°C."
    except ValueError:
        return "Error: Invalid input. Please provide a valid number."
    
tools = [
    Tool(
        name="MathExpressionCalculator",
        func=eval_expression,
        description="Safely evaluate full math expressions like '(8 + 2) * (5 - 3) / 2'. Input must be a valid math expression string.",
        return_direct=True,
    ),
    Tool(
        name="DaysPassedCalculator",
        func=days_passed_calculation,
        description=(
            "Use for questions like 'how many days since 1995-01-01', or date difference questions. "
            "Input must be in 'YYYY-MM-DD' format."
        ),
        return_direct=True,
    ),
    #Tool(
    #    name="ConversionFtoC",
    #    func=fahrenheit_to_celsius,
    #    description="Convert Fahrenheit to Celsius. Input must be a valid number representing temperature in Fahrenheit.",
    #    return_direct=True,
    #)
]

# --- Prompt Template ---
prompt = PromptTemplate(
    input_variables=["input"],
    template=(
        "You are a helpful and precise AI assistant.\n\n"
        "Use ONLY the MathExpressionCalculator tool for solving math expressions, like '(10 / 2) + 4 * (1 + 1)'.\n"
        "Use ONLY the DaysPassedCalculator tool for questions such as 'how many days since 1995-01-01'.\n"
        "Use ONLY the ConversionFtoC tool when the user asks to convert Fahrenheit to Celsius.\n\n"
        "Use ConversionFtoC if the input contains terms like: '°F', 'degrees Fahrenheit', 'deg F', or 'Fahrenheit'.\n"
        "NEVER use MathExpressionCalculator for dates or temperatures.\n"
        "NEVER use DaysPassedCalculator or ConversionFtoC for general math.\n"
        "NEVER use ConversionFtoC for dates.\n\n"
        "When using ConversionFtoC, pass only the number (e.g., 98.6) and ignore any text.\n\n"
        "Input: {input}\n\n"
        "{agent_scratchpad}"
    )
)

# --- LLM and Agent Setup ---
llm = ChatOllama(model="llama3.2:1b", temperature=0.0)  # Using Ollama's Llama 3.2 model
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)  # Uncomment to use OpenAI's GPT-3.5-turbo model
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True,
    )

# --- Running the Agent ---
result1 = executor.invoke({"input": "Compute the result of: (10 / 2) + 4 * (1 + 1)"})
print("Result: ", result1["output"])

result2 = executor.invoke({"input": "Calculate the result of: (8 + 2) * (5 - 3) / 2"})
print("Result 2: ", result2["output"])

result3 = executor.invoke({"input": "How many days have passed since 1995-01-01?"})
print("Result 3: ", result3["output"])

#result4 = executor.invoke({"input": "Convert 98.6"})
#print("Result 4: ", result4["output"])